In [2]:
import torch
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from gtts import gTTS
import timm
import warnings
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
warnings.filterwarnings("ignore")

In [3]:
classification_model = load_model('exp2_checkpoints/')
model_type = "MiDaS_small"
midas = torch.hub.load("intel-isl/MiDaS", model_type)

Using cache found in C:\Users\Admin/.cache\torch\hub\intel-isl_MiDaS_master
Using cache found in C:\Users\Admin/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master


Loading weights:  None


In [4]:
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform
    

Using cache found in C:\Users\Admin/.cache\torch\hub\intel-isl_MiDaS_master


In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

MidasNet_small(
  (pretrained): Module(
    (layer1): Sequential(
      (0): Conv2dSameExport(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
      (3): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU6(inplace=True)
          (se): Identity()
          (conv_pw): Conv2d(32, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act2): Identity()
        )
      )
      (4): Sequential(
        (0): InvertedResidual(
          (conv_pw): Conv2d(24, 144, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(144,

In [5]:
filename = 'yolov5/NUS_Cycle.jpeg'

In [10]:
img = cv2.imread(filename)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

input_batch = transform(img).to(device)

In [11]:
with torch.no_grad():
    prediction = midas(input_batch)

    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=img.shape[:2],
        mode="bicubic",
        align_corners=False,
    ).squeeze()

output = prediction.cpu().numpy()

In [12]:
def split_depthmap(depthmap):
    
    width = depthmap.shape[1]
    height = depthmap.shape[0]
    
    depthmap = depthmap[:int(height*0.7),:]
    
    max_depth_point = (np.where(depthmap== np.max(depthmap)))[1][0]
    
    if max_depth_point <= (width//3):
        return "left"
    elif (width//3) < max_depth_point <= (width * 2/3):
        return "middle"
    else:
        return "right"
       

In [13]:
def split_image(image, side):
    
    width = image.shape[1]
    height = image.shape[0]
    
    left_side = image[:,:width//3,:]
    middle = image[:,width//3:2*(width//3),:]
    right_side = image[:,2*(width//3):,:]
    
    if side == "left":
        return left_side
    elif side == "middle":
        return middle
    else:
        return right_side

In [10]:
def give_alert(img, depthmap, img_path):
    
    aoi_region = split_depthmap(depthmap)     ###WHICH SIDE OBJECT IS ON
    
    aoi_image = split_image(img, aoi_region)
    
#     val_test_gen = ImageDataGenerator(rescale = 1./255)


#     val_gen = val_test_gen.flow_from_directory(img,
#                                       target_size = (359, 426),
#                                       batch_size = 1,
#                                       shuffle = False,
#                                       color_mode = 'grayscale',
#                                       class_mode = 'categorical')

    img = img_to_array(img,  = (359, 426))
    img = img.reshape(img.shape[0], img.shape[1], img.shape[2], 1)

    
    preds = classification_model.predict(img)
    
    preds = to_categorical(preds.argmax(axis = 1))
    
    labels = ['animal', 'bench', 'bike', 'car', 'fire_hydrant', 'person', 'sign']
    
    obj = labels[preds.argmax()]
    
    if results.pandas().xyxy[0].empty:
        ##GIVE AUDIO ALERT FOR UNKNOWN OBJECT at detected side
        to_return = "Unknown object at {} side".format(aoi_region) 
        return to_return
    else:
        obj = results.pandas().xyxy[0].iloc[0]['name']
        ## GIVE AUDIO ALERT FOR obj and side    
        to_return = "{} at {} side".format(obj, aoi_region)
        return to_return

SyntaxError: invalid syntax (<ipython-input-10-d060073dd026>, line 17)

In [60]:
def pipeline(img_path):
    
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    input_batch = transform(img).to(device)
    
    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

    output = prediction.cpu().numpy()
    
    return give_alert(img, output, img_path)

In [61]:
pipeline(filename)

NameError: name 'give_alert' is not defined

In [24]:
filename = 'single_image//hydrant.jpg'
img = load_img(filename)
img = img_to_array(img)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
res = cv2.resize(img, dsize=(359,426), interpolation=cv2.INTER_CUBIC)

In [25]:
res.shape

(426, 359)

In [26]:
res = res.reshape(1, res.shape[0], res.shape[1], 1)

In [27]:
res.shape

(1, 426, 359, 1)

In [28]:
pred = classification_model.predict(res)

1/1 [==============================] - 0s 5ms/step


In [29]:
pred

array([[0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
plt.imshow(filename)

In [ ]:
test_gen = ImageDataGenerator()
predictions = classification_model.predict_generator(test_gen.flow(res))

In [20]:
classification_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, None, None, 32)    320       
                                                                 
 batch_normalization (BatchN  (None, None, None, 32)   128       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, None, None, 32)   0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 64)    18496     
                                                                 
 batch_normalization_1 (Batc  (None, None, None, 64)   2